In [ ]:
%load_ext autoreload
%autoreload 2
import os
# Set environment variables
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".99"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
# from jax import config
# config.update('jax_enable_x64', True)
# Check JAX configuration
devices = jax.local_devices()
print(devices)
print(jax.default_backend())
print(jax.devices())
import sys
# Add custom path
sys.path.append("/pchem-data/meuwly/boittier/home/pycharmm_test/src")
# from model import EF
# from loss import dipole_calc
# from  training import train_model # from model import dipole_calc
import jax
import optax
import e3x
# from dcmnet.analysis import create_model_and_params
# from data import prepare_batches, prepare_datasets
import numpy as np
from physnetjax.model import EF
from physnetjax.loss import dipole_calc
from  physnetjax.training import train_model # from model import dipole_calc
from physnetjax.data import prepare_batches, prepare_datasets
import orbax
from orbax.checkpoint import PyTreeCheckpointer
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
import jax
import jax.numpy as jnp
import flax.linen as nn
import networkx as nx
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import os
# Set environment variables
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".99"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
# from jax import config
# config.update('jax_enable_x64', True)
# Check JAX configuration
devices = jax.local_devices()
print(devices)
print(jax.default_backend())
print(jax.devices())
import sys
# Add custom path
sys.path.append("/pchem-data/meuwly/boittier/home/pycharmm_test/src")
# from model import EF
# from loss import dipole_calc
# from  training import train_model # from model import dipole_calc
import jax
import optax
import e3x
# from dcmnet.analysis import create_model_and_params
# from data import prepare_batches, prepare_datasets
import numpy as np
from physnetjax.model import EF
from physnetjax.loss import dipole_calc
from  physnetjax.training import train_model # from model import dipole_calc
from physnetjax.data import prepare_batches, prepare_datasets
import orbax
from orbax.checkpoint import PyTreeCheckpointer
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
# from physnetjax.analysis import *
import polars as pl
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tensorflow.python.summary.summary_iterator import summary_iterator

In [8]:
from openqdc.datasets import Spice
from physnetjax.savepad import * 

NATOMS = 96

ds = Spice(
    energy_unit="ev",
    distance_unit="ang",
    # array_format = "jax"
)
ds.read_preprocess()
datadicts = [dict(ds[_]) for _ in ds.subsample(5000)]
output1 = process_in_memory(datadicts, max_atoms=NATOMS)
datadicts = [dict(ds[_]) for _ in ds.subsample(500)]
output2 = process_in_memory(datadicts, max_atoms=NATOMS)

2024-12-08 15:49:42.038 | INFO     | openqdc.datasets.base:read_preprocess:435 - Reading preprocessed data.
2024-12-08 15:49:42.039 | INFO     | openqdc.datasets.base:read_preprocess:436 - Dataset spice with the following units:
                     Energy: hartree,
                     Distance: bohr,
                     Forces: hartree/bohr
2024-12-08 15:49:43.424 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded atomic_inputs with shape (33175288, 5), dtype float32
2024-12-08 15:49:43.425 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded position_idx_range with shape (1110165, 2), dtype int32
2024-12-08 15:49:43.425 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded energies with shape (1110165, 1), dtype float64
2024-12-08 15:49:43.426 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded forces with shape (33175288, 3, 1), dtype float32
2024-12-08 15:49:43.426 | INFO     | openqdc.datasets.base:read_preprocess:452 - Loaded n_atom

In [9]:
type(datadicts)

list

In [10]:
data_key, train_key = jax.random.split(
    jax.random.PRNGKey(42), 2)

In [5]:
from physnetjax.model import EF
from physnetjax.training import train_model 


In [ ]:
DEFAULT_DATA_KEYS = ["Z", "R", "D", "E", "F", "N"]

model = EF(natoms=NATOMS, charges=False, zbl=False)

params = train_model(
    train_key,
    model,
    output1,
    output2,
    num_epochs=int(1e6),
    learning_rate=0.001,
    energy_weight=NATOMS,
    #charges_weight=1,
    #forces_weight=100,
    schedule_fn="constant",
    optimizer="amsgrad",
    batch_size=10,
    num_atoms=NATOMS,
    data_keys=DEFAULT_DATA_KEYS,
#    restart=restart,
    print_freq=1,
    objective="valid_loss",
    best=1e6,
    )

Training Routine
Optimizer:
  init: <function chain.<locals>.init_fn at 0x153b86baede0> init_fn
  update: <function chain.<locals>.update_fn at 0x153b86baee80> update_fn
Transform:
  init: <function reduce_on_plateau.<locals>.init_fn at 0x153b86baef20> init_fn
  update: <function reduce_on_plateau.<locals>.update_fn at 0x153b86baf1a0> update_fn
Schedule_fn: <function constant_schedule.<locals>.<lambda> at 0x153b86bae2a0> <lambda>
learning_rate: 0.001
energy_weight: 96
forces_weight: 52.91772105638412
dipole_weight: 27.211386024367243
charges_weight: 14.399645351950548
batch_size: 10
num_atoms: 96
restart: False
conversion: {'energy': 23.060548012069496, 'forces': 23.060548012069496}
print_freq: 1
name: test
best: 1000000.0
objective: valid_loss
data_keys: ['Z', 'R', 'D', 'E', 'F', 'N']
ckpt_dir: /pchem-data/meuwly/boittier/home/pycharmm_test/ckpts
Train data keys: dict_keys(['Z', 'N', 'R', 'F', 'E'])
Valid data keys: dict_keys(['Z', 'N', 'R', 'F', 'E'])
Objective: valid_loss
Saving a r

I0000 00:00:1733669490.307214  771775 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 809 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:03:00.0, compute capability: 8.6


epoch:   1				 train:   valid:
    loss		[a.u.]     	 809838.688  1210851.000 1000000.000
    energy mae		[kcal/mol]	 15142.620  25875.168
    forces mae		[kcal/mol/Å]	 265.690   17.192
scale: 1.000000 0.001000 LR=0.001000000
dict_keys(['Z', 'N', 'R', 'F', 'E', 'dst_idx', 'src_idx', 'batch_mask', 'batch_segments', 'atom_mask'])
epoch:   2				 train:   valid:
    loss		[a.u.]     	 617887.938  1138491.500 1000000.000
    energy mae		[kcal/mol]	 11070.376  24403.344
    forces mae		[kcal/mol/Å]	 381.804   25.858
scale: 1.000000 0.001000 LR=0.001000000
dict_keys(['Z', 'N', 'R', 'F', 'E', 'dst_idx', 'src_idx', 'batch_mask', 'batch_segments', 'atom_mask'])
/pchem-data/meuwly/boittier/home/pycharmm_test/ckpts/test-ecdfaea0-2902-403e-8f44-27aba18a1c44/epoch-3
best!
epoch:   3				 train:   valid:
    loss		[a.u.]     	 500815.188  959244.062 959244.062
    energy mae		[kcal/mol]	 8967.378  20791.652
    forces mae		[kcal/mol/Å]	 410.616   46.433
scale: 1.000000 0.001000 LR=0.001000000
dict_key

In [ ]:
files = ["/pchem-data/meuwly/boittier/home/cf3criegee_27887.npz"]
train_data, valid_data = prepare_datasets(data_key, 25877, 2000,
                                          files, 
                                          clip_esp=False, natoms=8, clean=False)

In [383]:
train_data["R"].shape, output["R"].shape

((25877, 8, 3), (10, 100, 3))

In [384]:
train_data["Z"].shape, output["Z"].shape

((25877, 8), (10, 100))

In [413]:
train_data["N"].shape, output["N"].shape

((25877, 1), (10, 1))

In [390]:
train_data.keys()

dict_keys(['R', 'Z', 'F', 'E', 'N', 'D', 'Q'])

In [412]:
DEFAULT_DATA_KEYS = ["Z", "R", "D", "E", "F", "N"]

params = train_model(
    train_key,
    model,
    train_data,
    train_data,
    num_epochs=int(1e6),
    learning_rate=0.001,
    energy_weight=NATOMS,
    #charges_weight=1,
    #forces_weight=100,
    schedule_fn="constant",
    optimizer="amsgrad",
    batch_size=1,
    num_atoms=8,
    data_keys=DEFAULT_DATA_KEYS,
#    restart=restart,
    print_freq=1,
    objective="valid_loss",
    best=1e6,
    )

Training Routine
Optimizer:
  init: <function chain.<locals>.init_fn at 0x14c1b18432e0> init_fn
  update: <function chain.<locals>.update_fn at 0x14c1b18431a0> update_fn
Transform:
  init: <function reduce_on_plateau.<locals>.init_fn at 0x14c1b1843240> init_fn
  update: <function reduce_on_plateau.<locals>.update_fn at 0x14c1b1843880> update_fn
Schedule_fn: <function constant_schedule.<locals>.<lambda> at 0x14c1b1841300> <lambda>
learning_rate: 0.001
energy_weight: 200
forces_weight: 52.91772105638412
dipole_weight: 27.211386024367243
charges_weight: 14.399645351950548
batch_size: 1
num_atoms: 8
restart: False
conversion: {'energy': 23.060548012069496, 'forces': 23.060548012069496}
print_freq: 1
name: test
best: 1000000.0
objective: valid_loss
data_keys: ['Z', 'R', 'D', 'E', 'F', 'N']
ckpt_dir: /pchem-data/meuwly/boittier/home/pycharmm_test/ckpts
Train data keys: dict_keys(['R', 'Z', 'F', 'E', 'N', 'D', 'Q'])
Valid data keys: dict_keys(['R', 'Z', 'F', 'E', 'N', 'D', 'Q'])
Objective: va

TypeError: 'NoneType' object is not subscriptable